In [ ]:
import keras
import math
import pandas as pd
import numpy as np
from keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error
import time
from torch.utils.data import Dataset, DataLoader
import pickle
pd.set_option('display.max_rows', 500)
import os
import tensorflow as tf
import torch
import torch.nn as nn
from math import sqrt
# import rmse from sklearn
from sklearn.metrics import mean_squared_error


# define random seeds for Neural Networks
torch.manual_seed(0)
np.random.seed(0)
tf.random.set_seed(0)
# ignore warnings jupyter notebook
import warnings
warnings.filterwarnings('ignore')

# OWRI FRAMEWORK

In [ ]:
# results save path
base_result_path = '../results/METR-LA/LSTM'
exp_name = 'multivariate_AE_weighted_vector_real_time_results.pkl'
results_save_path = os.path.join(base_result_path, exp_name)

In [ ]:
with open('../data/METR-LA/METR_OWRI/featured_fpds_raw.pickle', 'rb') as f:
    results = pickle.load(f)

In [ ]:
# load data of correlated results from pickle file
with open('../results/METR-LA/outlier_scores/AE/correlated_results.pickle', 'rb') as f:
    correlated_results = pickle.load(f)

In [ ]:
def merge_trejectory_data(results, trajectory, direction):
    data = pd.DataFrame()
    for intersection_name in results[trajectory][direction]['raw']:
        intersection = results[trajectory][direction]['raw'][intersection_name]
        intersection = intersection.rename(columns={"cars": intersection_name})
        intersection = intersection.set_index(pd.DatetimeIndex(intersection['timestamp']))
        intersection = intersection.drop(columns=['timestamp'])
        data = pd.merge(data, intersection, left_index=True, right_index=True, how='outer')
    data.dropna(inplace=True)
    return data

In [ ]:
def merge_trejectory_data_metr(results):
    data = pd.DataFrame()
    for intersection_name in results['raw']:
        intersection = results['raw'][intersection_name]
        intersection = intersection.rename(columns={"cars": intersection_name})
        intersection = intersection.set_index(pd.DatetimeIndex(intersection['timestamp']))
        intersection = intersection.drop(columns=['timestamp'])
        data = pd.merge(data, intersection, left_index=True, right_index=True, how='outer')
    data.dropna(inplace=True)
    return data

In [ ]:
def add_top_corr_features(data, top_corr_df):
    # using numpy broadcasting
    corr_array = top_corr_df.values
    corr_array = corr_array.reshape(1,1,corr_array.shape[0],corr_array.shape[1])
    


    # using loops
    # arr1 = []
    # for i in range(data.shape[0]):
    #     arr2 = []
    #     for j in range(data.shape[1]):
    #         temp = data[i][j].reshape(-1,1) * top_corr_df.values
    #         temp = temp.reshape(-1)
    #         arr2.append(temp)
    #     arr1.append(np.array(arr2))
    # return np.array(arr1)

In [ ]:
def preprocess_df(df,top_corr_df, n_obs, n_features, sequence_length):
    #do scaling:
    scaler = StandardScaler()
    train_portion = 0.8
    test_portion = 0.2
    df_train = df[:math.ceil(len(df)*train_portion)].values
    df_test = df[math.ceil(len(df)*(train_portion)):].values
    train_X, train_y = df_train[:, :n_obs], df_train[:, -n_features]
    test_X, test_y = df_test[:, :n_obs], df_test[:, -n_features]
    # scl = scaler.fit(train_X) # fit only on training data
    # train_X = scl.transform(train_X)
    # test_X = scl.transform(test_X)
    train_X = train_X.reshape((train_X.shape[0], sequence_length, n_features))
    test_X = test_X.reshape((test_X.shape[0], sequence_length, n_features))

    # # add top correlated features weighted sum
    # train_X = add_top_corr_features(train_X, top_corr_df)
    # test_X = add_top_corr_features(test_X, top_corr_df)


    # # add top correlated features as it is 
    # corr_array = top_corr_df.values
    # corr_tiled_train = np.tile(corr_array, (train_X.shape[0], sequence_length, 1, 1))
    # corr_tiled_test = np.tile(corr_array, (test_X.shape[0], sequence_length, 1, 1))
    # train_X = np.concatenate([train_X[:, :, :, np.newaxis], corr_tiled_train], axis=3) # add outlier dimension
    # train_X = train_X.reshape(train_X.shape[0],train_X.shape[1],-1) # reshape to 3D
    # test_X = np.concatenate([test_X[:, :, :, np.newaxis], corr_tiled_test], axis=3) # add outlier dimension
    # test_X = test_X.reshape(test_X.shape[0],test_X.shape[1],-1) # reshape to 3D

    return train_X, train_y, test_X, test_y, scaler

In [ ]:
def reshape_df(multivariate_dict):
    train_X = c = np.stack(multivariate_dict['train_X'], axis=-2)
    train_y = c = np.stack(multivariate_dict['train_y'], axis=1)
    test_X = c = np.stack(multivariate_dict['test_X'], axis=-2)
    test_y = c = np.stack(multivariate_dict['test_y'], axis=1)
    train_X = train_X.reshape(train_X.shape[0],train_X.shape[1],-1) # reshape to 3D
    test_X = test_X.reshape(test_X.shape[0],test_X.shape[1],-1) # reshape to 3D
    return train_X, train_y, test_X, test_y

In [ ]:
def scale_df(train_X,test_X):
    scaler = StandardScaler()
    train_X = scaler.fit_transform(train_X.reshape(-1, train_X.shape[-1])).reshape(train_X.shape)
    test_X = scaler.transform(test_X.reshape(-1, test_X.shape[-1])).reshape(test_X.shape)
    return train_X, test_X, scaler

In [ ]:
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
	n_vars = 1 if type(data) is list else data.shape[1]
	df = pd.DataFrame(data)
	cols, names = list(), list()
	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(df.shift(i))
		names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
	# forecast sequence (t, t+1, ... t+n)
	for i in range(0, n_out):
		cols.append(df.shift(-i))
		if i == 0:
			names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
		else:
			names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
	# put it all together
	agg = pd.concat(cols, axis=1)
	agg.columns = names
	# drop rows with NaN values
	if dropnan:
		agg.dropna(inplace=True)
	return agg

In [ ]:
def get_top_corr_features(top_corr_df, top_k_col):
    # create new coorelation df with top k correlated intersections for each intersection
    new_corr_df = []
    for col in top_corr_df.columns:
        new_corr_df.append(top_corr_df[col].sort_values(ascending=False)[:top_k_col].values)
    new_corr_df = pd.DataFrame(new_corr_df, index=top_corr_df.columns)
    return new_corr_df

In [ ]:
class LSTM_uni(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, device = 'mps',layer_dim=1, dropout_prob = 0.2):
        super(LSTM_uni, self).__init__()
        self.hidden_dim = hidden_dim # number of hidden units in hidden state
        self.layer_dim = layer_dim # number of stacked lstm layers
        # batch_first=True causes input/output tensors to be of shape
        # (batch_dim, seq_dim, feature_dim)
        self.lstm = nn.LSTM(input_dim, hidden_dim, layer_dim, batch_first=True, dropout=dropout_prob)
        self.fc = nn.Linear(hidden_dim, output_dim) # fully connected layer

    def forward(self, x, future=False):
        # input x is expected to be of shape (batch_dim, seq_dim, feature_dim)
        # hidden and cell states are expected along with input x in LSTMs = (h_0, c_0)
        # Initialize hidden state with zeros (layer_dim, batch_size, hidden_dim)
        h0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim, device=device).requires_grad_()
        c0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim, device=device).requires_grad_()
        # LSTM output is Outputs: output, (h_n, c_n)
        # output is of shape (batch_dim, seq_dim, hidden_dim), h_n and c_n are of shape (layer_dim, batch_dim, hidden_dim)
        out, (hn, cn) = self.lstm(x, (h0.detach(), c0.detach()))
        out = out[:, -1, :] # only take the last output of the sequence
        out = self.fc(out) # fully connected layer
        return out

In [ ]:
def train_model(model, train_X,train_y, loss_fn, optimiser, device, batch_size, epochs=250):
    history = {}
    history['train_loss'] = []

    train_X_loader = DataLoader(train_X, batch_size=batch_size, shuffle=False)
    train_y_loader = DataLoader(train_y, batch_size=batch_size, shuffle=False)

    for epoch in range(epochs):
        history[epoch] = []
        ep_start = time.time()
        running_loss = 0.0
        for bx, data in enumerate(zip(train_X_loader,train_y_loader)):
            X = data[0].to(device)
            y = data[1].to(device)
            bt = model(X)
            loss = loss_fn(bt.reshape(-1), y.reshape(-1)) # calculate loss for input and recreated output
            history[epoch].append(loss.item())
            optimiser.zero_grad()
            loss.backward()
            optimiser.step()
            running_loss += loss.item()
        epoch_loss = running_loss/train_X.shape[0]
        history['train_loss'].append(epoch_loss)

    return history
    

In [ ]:
def model_evaluation( model, test_X, device):
    test_X_loader = DataLoader(test_X, batch_size=64, shuffle=False)
    model = model.eval()
    preds = []
    with torch.no_grad():
        for bx, data in enumerate(test_X_loader):
            X = data.to(device)
            bt = model(X)
            preds.append(bt.cpu().numpy())
    preds = np.vstack(preds)
    return preds

In [ ]:
def preprocess_df_2(df,ae_score):
    weighted_ls = []
    correlated_AE = ae_score.corr().values
    for idx, row in df.iterrows():
        weighted_ls.append(np.sum(row.values*correlated_AE, axis=1))
    weighted_df = pd.DataFrame(weighted_ls, columns=df.columns)
    return weighted_df

In [ ]:
# declare variables
# thresholds = [0,0.25,0.5,0.75,1]
thresholds = [0, 0.05, 0.1, 0.15]
epoch = 100
batch_size = 64
learning_rate = 0.1
hidden_size = 32
num_layers = 1
dropout = 0.2
sequence_length = 12
output_pred = 1 # number of time steps to predict
device = 'mps' if torch.backends.mps.is_available() else 'cpu'

In [ ]:
errors={}  # define a dictionary to store the errors
dfs={} # Define a dictionary to store the dataframes
errors = {} # Define a dictionary to store the errors
intersection_arrays = [] # Define a list to store the intersection arrays
for threshold in thresholds:
    print("Starting threshold: {}".format(threshold))
    # Define a dictionary to store the errors for this trajectory, direction, and threshold
    errors[threshold]={}
    # ------------------------------------ data processing ---------------------------------------- #
    ae_score = correlated_results['df'] # AE scores of the current trajectory and direction
    df = merge_trejectory_data_metr(results)# get raw data of the current trajectory and direction
    df = df[ae_score.columns.to_list()]
    number_of_cols = math.ceil(len(ae_score.columns)*threshold) # number of outlier weighted intersections to use
    if number_of_cols==0: # if threshold is 0, then use the target intersection only
        number_of_cols=1
    
    multivariate_dict = {
        'train_X':[],'train_y':[],'test_X':[],'test_y':[]
    }
    for target in df.columns:
        top_corr_df = ae_score.corr()[target].sort_values(ascending=False)[:number_of_cols] # get the top correlated intersections
        isct_inc = top_corr_df.index.tolist()
        df_temp = df[isct_inc].copy(deep=True)
        df_temp = df_temp.mul(top_corr_df, axis=1)
        df_temp = df_temp.astype('float32')
        sequence_length = 12 # number of time steps to look back
        n_features = len(isct_inc) # number of features (correlated intersections)
        output_pred = 1 # number of time steps to predict
        n_obs = sequence_length * n_features # number of columns in the input
        reframed = series_to_supervised(df_temp, sequence_length, output_pred)
        train_X, train_y, test_X, test_y, scl = preprocess_df(reframed, top_corr_df, n_obs, n_features, sequence_length)
        train_X, train_y, test_X, test_y = train_X.astype('float32'), train_y.astype('float32'), test_X.astype('float32'), test_y.astype('float32')
        multivariate_dict['train_X'].append(train_X)
        multivariate_dict['train_y'].append(train_y)
        multivariate_dict['test_X'].append(test_X)
        multivariate_dict['test_y'].append(test_y)
    
    # reshape the data for training and testing
    train_X, train_y, test_X, test_y = reshape_df(multivariate_dict)
    del multivariate_dict # delete the dictionary to save memory
    train_X, test_X, scaler = scale_df(train_X, test_X)

    # # # ------------------------------------ modelling ---------------------------------------------- #
    # define model, loss function and optimizer
    model = LSTM_uni(input_dim = train_X.shape[-1], hidden_dim = hidden_size, layer_dim = num_layers, output_dim = train_y.shape[1], dropout_prob= dropout)
    model = model.to(device)
    loss_fn = torch.nn.MSELoss()
    optimiser = torch.optim.Adam(model.parameters(), lr=learning_rate)
    start = time.time()
    history = train_model(model, train_X,train_y, loss_fn, optimiser, device, batch_size = batch_size, epochs=epoch)
    end = time.time()
    print("Training time: {}".format(end-start))

    # ------------------------------------ evaluation ---------------------------------------------- #
    yhat = model_evaluation(model, test_X , device)
    errors[threshold]['RMSE'] = sqrt(mean_squared_error(yhat,test_y))
    errors[threshold]['MAE'] = mean_absolute_error(yhat,test_y)
    errors[threshold]['history'] = history
    errors[threshold]['df'] = {"Real":test_y,"Predicted":yhat}
    errors[threshold]['train_time'] = end-start
    print("RMSE: {}".format(errors[threshold]['RMSE']))


# save errors in save path as pickle file
with open(results_save_path, 'wb') as handle:
    pickle.dump(errors, handle)

In [ ]:
res

In [ ]:
# errors={}  # define a dictionary to store the errors
# dfs={} # Define a dictionary to store the dataframes
# errors = {} # Define a dictionary to store the errors
# intersection_arrays = [] # Define a list to store the intersection arrays
# for trajectory in results.keys(): # Loop over all trajectories
#     print("\nStarting trajectory: {}".format(trajectory))  
#     # Define a dictionary to store the errors for this trajectory
#     errors[trajectory]={}
#     # Loop over all directions
#     for direction in results[trajectory]:
#         print("\nStarting direction: {}".format(direction))
#         # Define a dictionary to store the errors for this trajectory and direction
#         errors[trajectory][direction]={}
#         # Loop over all thresholds
#         for threshold in thresholds:
#             print("Starting threshold: {}".format(threshold))
#             # Define a dictionary to store the errors for this trajectory, direction, and threshold
#             errors[trajectory][direction][threshold]={}
#             # ------------------------------------ data processing ---------------------------------------- #
#             df = merge_trejectory_data(results, trajectory, direction)# get raw data of the current trajectory and direction
#             ae_score = correlated_results[trajectory][direction] # AE scores of the current trajectory and direction
#             top_k_col = math.ceil(len(ae_score.columns)*threshold) # number of outlier weighted intersections to use
#             if top_k_col==0: # if threshold is 0, then use the target intersection only
#                 top_k_col=1
#             top_corr_df = ae_score.corr()[df.columns.to_list()] # rearrange the correlation matrix
#             top_corr_df = get_top_corr_features(top_corr_df, top_k_col) # get the top k correlated intersections
#             n_features = len(df.columns) # number of features (correlated intersections)
#             n_obs = sequence_length * n_features # number of columns in the input
#             # weighted_df = preprocess_df_2(df,ae_score)
#             reframed = series_to_supervised(df, sequence_length, output_pred)
#             train_X, train_y, test_X, test_y, scl = preprocess_df(reframed,top_corr_df, n_obs, n_features, sequence_length)
#             train_X, train_y, test_X, test_y = train_X.astype('float32'), train_y.astype('float32'), test_X.astype('float32'), test_y.astype('float32')


#             # # # ------------------------------------ modelling ---------------------------------------------- #
#             # define model, loss function and optimizer
#             model = LSTM_uni(input_dim = train_X.shape[-1], hidden_dim = hidden_size, layer_dim = num_layers, output_dim = train_y.shape[1], dropout_prob= dropout)
#             model = model.to(device)
#             loss_fn = torch.nn.MSELoss()
#             optimiser = torch.optim.Adam(model.parameters(), lr=learning_rate)
#             start = time.time()
#             history = train_model(model, train_X,train_y, loss_fn, optimiser, device, batch_size = batch_size, epochs=epoch)
#             end = time.time()
#             print("Training time: {}".format(end-start))

#             # ------------------------------------ evaluation ---------------------------------------------- #
#             yhat = model_evaluation(model, test_X , device)
#             errors[trajectory][direction][threshold]['RMSE'] = sqrt(mean_squared_error(yhat,test_y))
#             errors[trajectory][direction][threshold]['MAE'] = mean_absolute_error(yhat,test_y)
#             errors[trajectory][direction][threshold]['history'] = history
#             errors[trajectory][direction][threshold]['df'] = {"Real":test_y,"Predicted":yhat}
#             errors[trajectory][direction][threshold]['train_time'] = end-start
#             print("RMSE: {}".format(errors[trajectory][direction][threshold]['RMSE']))


# # save errors in save path as pickle file
# with open(results_save_path, 'wb') as handle:
#     pickle.dump(errors, handle)

In [ ]:
c

In [ ]:
a

In [ ]:
b

In [ ]:
multivariate_dict['769346']

In [ ]:
ae_score.corr().head()

In [ ]:
df.head()

In [ ]:
# sort values by each column and get the top k and keep ramaining columns as NaN
def get_top_corr_features(df, k):
    df = df.apply(lambda x: x.sort_values(ascending=False).index)
    df = df.apply(lambda x: pd.Series(x[:k]))
    return df

In [ ]:
df_temp = ae_score.corr()

In [ ]:
# df = df.apply(lambda x: x.sort_values(ascending=False))

In [ ]:
# df['717508'].sort_values(ascending=False)

In [ ]:
sorted_df = df_temp.apply(lambda row: row.sort_values(ascending=False).iloc[:10]).replace(np.nan, 0)

In [ ]:
for idx, row in df_temp.iterrows():
    l = row.sort_values(ascending=False).iloc[:10].index.to_list()
    break

In [ ]:
df_temp.head()

In [ ]:
60.125*0.698464

In [ ]:
train_X[0][0]

In [ ]:
temp = train_X[0][0] * df_temp.values
temp

In [ ]:
temp.shape

In [ ]:
temp.T['717508'].sort_values(ascending=False)

In [ ]:
sorted_df['717508'].sort_values(ascending=False)

In [ ]:

for idx, row in df.iterrows():
    temp = row.sort_values(ascending=False)
    temp.iloc[2:] = 0
    print(temp)
    c+=1
    if c==3:
        break

In [ ]:
sorted_df.head()

In [ ]:
sorted_df['772151']

In [ ]:
df[ae_score.corr().columns.to_list()].head()

In [ ]:
train_X.shape

In [ ]:
temp = top_corr_df.values

In [ ]:
temp_2 = temp.reshape(1,1,temp.shape[0],temp.shape[1])

In [ ]:
temp_2.shape

In [ ]:
temp3 = train_X[:, :, :, np.newaxis] * temp_2

In [ ]:
temp4 = temp3.reshape(temp3.shape[0],temp3.shape[1],-1)

In [ ]:
temp

In [ ]:
# load data of correlated results from pickle file
with open('../results/hauge/LSTM/', 'rb') as f:
    errors = pickle.load(f)

In [ ]:
AE_results={}
for trajectory in errors.keys():
    for direction in errors[trajectory].keys():
        for threshold in errors[trajectory][direction].keys():
            AE_results[trajectory+'_'+direction+'_'+str(threshold)] = errors[trajectory][direction][threshold]['MAE']

In [ ]:
AE_results

In [ ]:
AE_results

In [ ]:
AE_results

In [ ]:
top_corr_df

In [ ]:
top_corr_df.shape

In [ ]:
temp = np.reshape(top_corr_df.values, (1, 1, 15, 8))

In [ ]:
temp.shape

In [ ]:
train_X.shape

In [ ]:
np.array(ls_big).shape

In [ ]:
np.dot(train_X[0][0].reshape(-1,1), top_corr_df.values)

In [ ]:
top_corr_df.values

In [ ]:
train_X

In [ ]:
# multiplying with the 3rd dimention of the whole train_X with shape (n_samples, n_timesteps, n_features)

In [ ]:
train_X.shape

In [ ]:
train_X[0][0]

In [ ]:
ae_score

In [ ]:
with open('../results/hauge/LSTM/multivariate_AE_weighted_real_time_results.pkl', 'rb') as f:
    errors = pickle.load(f)

In [ ]:
AE_results={}
for trajectory in errors.keys():
    for direction in errors[trajectory].keys():
        for threshold in errors[trajectory][direction].keys():
            AE_results[trajectory+'_'+direction+'_'+str(threshold)] = errors[trajectory][direction][threshold]['RMSE']

In [ ]:
AE_results

In [ ]:
temp1 = df.iloc[0:1].values
temp1

In [ ]:
top_corr_df

In [ ]:
weighted_ls = []
for idx, row in df.iterrows():
    weighted_ls.append(np.sum(row.values*ae_score.corr().values, axis=1))
    break
weighted_df = pd.DataFrame(weighted_ls, columns=df.columns)

In [ ]:
weighted_df

In [ ]:
# multiply top_corr_df and temp1 on axis 1
temp2 = np.multiply(top_corr_df, temp1.T)

In [ ]:
temp2

In [ ]:
temp = df.iloc[0:1].values* ae_score.values
temp

In [ ]:
# get sum of each row in temp
temp.sum(axis=1)

In [ ]:
AE_results

# EXTRA

In [ ]:
c = [a.cpu().detach().numpy(),b.cpu().detach().numpy()]

In [ ]:
d = np.vstack(c)

In [ ]:
a.shape

In [ ]:
temp = test_y[:64,:].shape

In [ ]:
test_y

In [ ]:
mean_squared_error(bt.cpu().detach().numpy(),y.cpu().detach().numpy())

In [ ]:
loss_fn(bt.reshape(-1),y.reshape(-1))

In [ ]:
train_X.shape

In [ ]:
model

In [ ]:
b = ae_score.corr().values
b.shape

In [ ]:
b_tiled = np.tile(b, (12, 1, 1))

In [ ]:
b_tiled.shape

In [ ]:
b_tiled_temp = np.tile(b, (78327, 12, 1, 1))

In [ ]:
b_tiled_temp.shape

In [ ]:
b_tiled_temp

In [ ]:
b_tiled.shape

In [ ]:
c = np.concatenate([train_X[:, :, :, np.newaxis], b_tiled_temp], axis=3)

In [ ]:
train_X.shape

In [ ]:
a.shape

In [ ]:
c[1]

In [ ]:
train_X[0]

In [ ]:
c = np.concatenate([a[:, :, np.newaxis], b_reshaped], axis=2)

In [ ]:
# stack each row of b with each element of a
a[0].shape

In [ ]:
np.hstack((a[0].reshape(-1,1),b))

In [ ]:
b

In [ ]:
# merge a and b to make it 12 x 15 x 15 array
c = np.concatenate((a,b), axis=0)

In [ ]:
c

In [ ]:
ae_score.corr().columns.to_list()

In [ ]:
ae_score.corr()[isct_inc]

In [ ]:
data.columns.to_list()

In [ ]:
data

In [ ]:
AE_results={}
for trajectory in errors.keys():
    for direction in errors[trajectory].keys():
        for threshold in errors[trajectory][direction].keys():
            AE_results[trajectory+'_'+direction+'_'+str(threshold)] = errors[trajectory][direction][threshold]['RMSE']

In [ ]:
AE_results